In [ ]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
import germansentiment

In [ ]:
# define stopwords
stop_words = open('stopwords.txt','r').read().split('\n')

In [ ]:
import csv

# import scraped data
d1 = pd.read_csv("Product_Hunt.csv")
d1 = d1['Review text']

d2 = pd.read_csv("Trustpilot.csv")
d2 = d2['Review text']

d = [d1, d2]
d = pd.concat(d)

In [ ]:
import gensim
from gensim.utils import simple_preprocess

# topic modeling
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data = d.values.tolist()
data_words = list(sent_to_words(data))

# remove stop words
data_words = remove_stopwords(data_words)

In [ ]:
import gensim.corpora as corpora

# create dictionary
id2word = corpora.Dictionary(data_words)

# create corpus
texts = data_words

# term document frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
from pprint import pprint

# define number of topics
num_topics = 5

# build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# print the keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os

# output graphic representation
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(LDAvis_prepared, "LDA_Reviews_EN_"+str(num_topics) +'.html')
LDAvis_prepared